In [172]:
#MODEL = "gpt-4o-2024-08-06"
#MODEL = "gpt-4o-mini"
MODEL = "gpt-3.5-turbo"
#prompt = [{"role": "user", "content": ""}]
loops = 100
file = f"{MODEL}/alphabet/5"

import os
from openai import OpenAI
import pandas as pd
from IPython.display import display_markdown
import copy

client = OpenAI()
df_path = os.path.join(os.getcwd(),"completions")
chat = False


def transform_roles(chat,assistant_transform_type = "user-assistant"):
    prompt_messages = copy.deepcopy(chat)
    if assistant_transform_type == "user-assistant":
      i = 1
      while i <= len(prompt_messages) and prompt_messages[-i]["role"] != "system":
        if i%2 == 1:
          prompt_messages[-i]["role"] = "user"
        else:
          prompt_messages[-i]["role"] = "assistant"
        i += 1
    else:
      for prompt_message in prompt_messages:
        if prompt_message["role"] == "assistant":
          prompt_message["role"] = assistant_transform_type
    return prompt_messages

In [173]:
# Load completions from json, if continuing with a file generated in the past

try:
    with open(os.path.join(df_path,file), "r") as f:
        completions = pd.read_json(f, lines=True)

# If not, create a new completions database

except FileNotFoundError:
    completions = pd.DataFrame([],columns=['Response', 'Eval', 'Data', 'Instruction'])
    print(completions)

Empty DataFrame
Columns: [Response, Eval, Data, Instruction]
Index: []


In [174]:
for i in range(loops):
    completion = client.chat.completions.create(
        messages = prompt,
        model = MODEL,
        )
    response = completion.choices[0].message.content

    data = {'Response': response, 'Eval': "", 'Data': {}, 'Instruction': prompt}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = {"Completion":completion.to_dict()}
    completions = pd.concat([completions, data],ignore_index=True)
    completions.to_json(os.path.join(df_path, file), lines=True, orient="records")

In [175]:
completions

,Response,Eval,Data,Instruction
0,"\n\nAdditionally, attending tech conferences, ...",,{'Completion': {'id': 'chatcmpl-9wmZ2v9gffVICj...,"[{'role': 'user', 'content': 'Stay up to date ..."
1,\n\nSome popular tech blogs to follow include ...,,{'Completion': {'id': 'chatcmpl-9wmZ5rNBJKJDwM...,"[{'role': 'user', 'content': 'Stay up to date ..."
2,Some popular tech blogs and websites to follow...,,{'Completion': {'id': 'chatcmpl-9wmZ7LLNzYsc2h...,"[{'role': 'user', 'content': 'Stay up to date ..."
3,Some ways to stay up to date with tech trends ...,,{'Completion': {'id': 'chatcmpl-9wmZAbqu5OLmy6...,"[{'role': 'user', 'content': 'Stay up to date ..."
4,Some popular tech blogs to follow include Tech...,,{'Completion': {'id': 'chatcmpl-9wmZCiMAqENGWf...,"[{'role': 'user', 'content': 'Stay up to date ..."
...,...,...,...,...
95,"Additionally, attending technology conferences...",,{'Completion': {'id': 'chatcmpl-9wmcebKVPpbyX7...,"[{'role': 'user', 'content': 'Stay up to date ..."
96,"Additionally, attending tech conferences and s...",,{'Completion': {'id': 'chatcmpl-9wmcguwws3Hw9E...,"[{'role': 'user', 'content': 'Stay up to date ..."
97,"\n\nAdditionally, attending tech events, confe...",,{'Completion': {'id': 'chatcmpl-9wmcizkG3AqW8V...,"[{'role': 'user', 'content': 'Stay up to date ..."
98,"\n\nAdditionally, attending industry conferenc...",,{'Completion': {'id': 'chatcmpl-9wmckmUh1b7xo4...,"[{'role': 'user', 'content': 'Stay up to date ..."


In [169]:
n = 80
completions.iloc[n]

Response       Stay up to date with trends in technology by f...
Eval                                                        Best
Data           {'Completion': {'id': 'chatcmpl-9wmJ6YRXimFRfp...
Instruction    [{'role': 'user', 'content': '

Ultimately, st...
Name: 80, dtype: object

In [168]:
completions.loc[n,"Eval"] = "Best"
completions.to_json(os.path.join(df_path, file), lines=True, orient="records")

In [170]:
tag = "Best"
selected_response = completions[completions["Eval"].apply(lambda x: x.startswith(tag))].iloc[0]["Response"]
if chat:
    messages = copy.deepcopy(completions[completions["Eval"].apply(lambda x: x.startswith(tag))].iloc[0]["Instruction"])
    messages.append({"role": "assistant", "content": selected_response_response})
    prompt = transform_roles(messages)
else:
    prompt = [{"role": "user", "content": selected_response}]
print(prompt)

[{'role': 'user', 'content': "Stay up to date with trends in technology by following tech blogs, subscribing to newsletters, and networking with industry professionals. It's also crucial to be willing to experiment with new technologies and be open to embracing change in order to stay relevant and innovative in today's fast-paced global market. By staying informed, adaptable, and forward-thinking, individuals and businesses can thrive in an ever-evolving technological world."}]
